In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import random
import datetime
import matplotlib.pyplot as plt
import pip
pip.main(['install','seaborn'])
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from keras.applications import DenseNet121, VGG19
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, LeakyReLU, GlobalAveragePooling2D, Conv2D, Input
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
modelclass = 'Female'
suffix = 'Female'
iterations = 5
iterations2 = 5

def CreateModel(learn_rate):
    input_tensor = Input(shape=(256, 256, 3))
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

    for layer in base_model.layers[:10]:
        layer.trainable = False
        
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024)(x)
    x = LeakyReLU(negative_slope=0.01)(x)
    x = Dropout(0.5)(x)
    predictions = Dense(5, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=learn_rate),
                  loss=BinaryCrossentropy(),
                  metrics=['accuracy'])
    return model

base_dir = 'Data/'
train_images = np.load(os.path.join(base_dir, 'train_images.npy'))
train_labels = np.load(os.path.join(base_dir, 'train_labels.npy'))
val_images = np.load(os.path.join(base_dir, 'val_images.npy'))
val_labels = np.load(os.path.join(base_dir, 'val_labels.npy'))

if len(train_images.shape) == 3:
    train_images = np.expand_dims(train_images, axis=-1)
    train_images = np.repeat(train_images, 3, axis=-1)

if len(val_images.shape) == 3:
    val_images = np.expand_dims(val_images, axis=-1)
    val_images = np.repeat(val_images, 3, axis=-1)

class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels.flatten())
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

print("Initial class weights:", class_weights)
print("Start time: ", datetime.datetime.now())

best_model_path = f'Models/class{suffix}{modelclass}temp.keras'
model_checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='min')

best_val_loss = float('inf')
for i in range(iterations):
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=1e-7)
    model = CreateModel(1e-5)
    print(f"Training iteration {i+1}/{iterations}")
    history = model.fit(train_images, train_labels,
                        epochs=50,
                        batch_size=16,
                        validation_data=(val_images, val_labels),
                        class_weight=class_weights,
                        callbacks=[early_stopping, reduce_lr, model_checkpoint])

    model.load_weights(best_model_path)
    val_loss = min(history.history['val_loss'])
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print(f"Best current val:{best_val_loss}")
        model.save(f'Models/class_{suffix}{modelclass}_overallbest.keras')

print("Training Complete")

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Requirement already satisfied: seaborn in /opt/conda/lib/python3.11/site-packages (0.13.2)

Requirement already satisfied: numpy!=1.24.0,>=1.20 in /opt/conda/lib/python3.11/site-packages (from seaborn) (1.26.4)

Requirement already satisfied: pandas>=1.2 in /opt/conda/lib/python3.11/site-packages (from seaborn) (2.2.3)

Requirement already satisfied: matplotlib!=3.6.1,>=3.4 in /opt/conda/lib/python3.11/site-packages (from seaborn) (3.9.3)

...

  1/161 ━━━━━━━━━━━━━━━━━━━━ 1:02:04 23s/step - accuracy: 0.5000 - loss: 0.6652

I0000 00:00:1738439765.838726    6936 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.3120 - loss: 0.5985

E0000 00:00:1738439816.716134    6937 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738439816.925134    6937 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738439817.530335    6937 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1738439817.812203    6937 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


161/161 ━━━━━━━━━━━━━━━━━━━━ 78s 343ms/step - accuracy: 0.3120 - loss: 0.5983 - val_accuracy: 0.3852 - val_loss: 0.4813 - learning_rate: 1.0000e-05
Epoch 2/50
161/161 ━━━━━━━━━━━━━━━━━━━━ 41s 213ms/step - accuracy: 0.3537 - loss: 0.5271 - val_accuracy: 0.3891 - val_loss: 0.4800 - learning_rate: 1.0000e-05
Epoch 3/50
161/161 ━━━━━━━━━━━━━━━━━━━━ 34s 214ms/step - accuracy: 0.3743 - loss: 0.5039 - val_accuracy: 0.4163 - val_loss: 0.4604 - learning_rate: 1.0000e-05
Epoch 4/50
161/161 ━━━━━━━━━━━━━━━━━━━━ 34s 214ms/step - accuracy: 0.4214 - loss: 0.4633 - val_accuracy: 0.4008 - val_loss: 0.4602 - learning_rate: 1.0000e-05
Epoch 5/50
161/161 ━━━━━━━━━━━━━━━━━━━━ 35s 214ms/step - accuracy: 0.4670 - loss: 0.4309 - val_accuracy: 0.4300 - val_loss: 0.4503 - learning_rate: 1.0000e-05
Epoch 6/50
161/161 ━━━━━━━━━━━━━━━━━━━━ 34s 214ms/step - accuracy: 0.4744 - loss: 0.4028 - val_accuracy: 0.4494 - val_loss: 0.4419 - learning_rate: 1.0000e-05
Epoch 7/50
161/161 ━━━━━━━━━━━━━━━━━━━━ 34s 214ms/step - 

In [2]:
best_model_path = f'Models/class{suffix}secondary{modelclass}temp.keras'
model_checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True, mode='min')
best_val_loss = float('inf')
for i in range (iterations2):
    model = CreateModel(1e-7)
    model.load_weights(f'Models/class_{suffix}{modelclass}_overallbest.keras')
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-8)
    
    print(f"Training iteration {i+1}/{iterations2}")
    history = model.fit(train_images, train_labels,
                        epochs=50,
                        batch_size=16,
                        validation_data=(val_images, val_labels),
                        class_weight=class_weights,
                        callbacks=[early_stopping, reduce_lr, model_checkpoint])

    model.load_weights(best_model_path)
    val_loss = min(history.history['val_loss'])
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print(f"Best last val:{val_loss}")
        print(f"Best current val:{best_val_loss}")
        model.save(f'Models/class_{suffix}{modelclass}_overallbestFIN.keras')

/opt/conda/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Training iteration 1/5
Epoch 1/50
161/161 ━━━━━━━━━━━━━━━━━━━━ 43s 237ms/step - accuracy: 0.6892 - loss: 0.2836 - val_accuracy: 0.5623 - val_loss: 0.3992 - learning_rate: 1.0000e-07
Epoch 2/50
161/161 ━━━━━━━━━━━━━━━━━━━━ 33s 207ms/step - accuracy: 0.6768 - loss: 0.2854 - val_accuracy: 0.5584 - val_loss: 0.4028 - learning_rate: 1.0000e-07
Epoch 3/50
161/161 ━━━━━━━━━━━━━━━━━━━━ 33s 208ms/step - accuracy: 0.6856 - loss: 0.2757 - val_accuracy: 0.5584 - val_loss: 0.4013 - learning_rate: 1.0000e-07
Epoch 4/50
161/161 ━━━━━━━━━━━━━━━━━━━━ 36s 223ms/step - accuracy: 0.6881 - loss: 0.2745 - val_accuracy: 0.5564 - val_loss: 0.4026 - learning_rate: 1.0000e-07
Epoch 5/50
161/161 ━━━━━━━━━━━━━━━━━━━━ 33s 208ms/step - accuracy: 0.6647 - loss: 0.2799 - val_accuracy: 0.5564 - val_loss: 0.4027 - learning_rate: 1.0000e-08
Epoch 6/50
161/161 ━━━━━━━━━━━━━━━━━━━━ 34s 208ms/step - accuracy: 0.6915 - loss: 0.2789 - val_accuracy: 0.5564 - val_loss: 0.4028 - learning_rate: 1.0000e-08
Best last val:0.3992293